In [54]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from tool.core import data_types
from tool.pyqt_gui.qt_utils import find_pkl

In [55]:
session_folder = '/home/nastya/Desktop/ood_datasets/DroneBird/oodsession_7'
ood_file_path = ood_file = find_pkl.get_ood_file(session_folder)
embeddings_file = find_pkl.get_embeddings_file(session_folder)

print(ood_file_path)
print(embeddings_file)

ood_df = pd.read_pickle(ood_file_path)
emb_df = pd.read_pickle(embeddings_file)

data_df = pd.merge(emb_df, ood_df[
                  [data_types.RelativePathType.name(), data_types.OoDScoreType.name()]],
                                    on=data_types.RelativePathType.name(), how='inner')

ood_score = data_df[data_types.OoDScoreType.name()].values

/home/nastya/Desktop/ood_datasets/DroneBird/oodsession_7/ood_score_230507_143431.445.ood.pkl
/home/nastya/Desktop/ood_datasets/DroneBird/oodsession_7/TimmResnetWrapper_densenet121_DroneBird_1024_230507_134347.030.emb.pkl


In [61]:
def get_predicted_label(probs, labels):
    return labels[np.argmax(probs)]

data_df["pred_label"] = data_df.apply(lambda row: get_predicted_label(row[data_types.ClassProbabilitiesType.name()], 
                                                                      row[data_types.LabelsType.name()]), axis=1).values

data_df["miss"] = data_df.apply(lambda row: row[data_types.LabelType.name()] != row["pred_label"], axis=1).values

data_df["conf"] = data_df.apply(lambda row: max(row[data_types.ClassProbabilitiesType.name()]), axis=1).values


In [57]:
ood_folders= ['drones/close_to_ood', 'birds/close_to_ood', 'birds/bird_and_person', 'bird_drone',
              'ood_samples'] 

def in_ood(img_path: str) -> bool:
    p, _ = os.path.split(img_path)
    return p in ood_folders

data_df["ood_flag"] = data_df.apply(lambda row: in_ood(row[data_types.RelativePathType.name()]), axis=1).values

In [58]:
n_samples = data_df.shape[0]
(data_df["ood_flag"][:n_samples].values).sum()

95

In [59]:
data_df.sort_values(by=[data_types.OoDScoreType.name()], inplace=True, ascending=False)
(data_df["ood_flag"][:50].values).sum()

11

In [60]:
data_df.sort_values(by=[data_types.OoDScoreType.name()], inplace=True, ascending=False)
(data_df["ood_flag"][:100].values).sum()

26

In [62]:
(data_df["miss"][:n_samples].values).sum()

129

In [66]:
(data_df[data_df["miss"]]["ood_score"].values > 0.75).sum()

116

In [69]:
data_df.loc[(data_df['miss']) & (data_df['ood_score'] > 0.75) & (data_df['conf'] > 0.75)].shape[0]

89

In [48]:
data_df[data_df["ood_flag"]].tail(10)

,relative_path,labels,test_sample,label,ood_score,ood_flag
693,drones/close_to_ood/e5ed9253-3142-4bc7-8ee3-5e...,"[Drone, Bird]",True,Drone,0.388912,True
1354,birds/close_to_ood/singleBirdinsky247.jpeg,"[Drone, Bird]",True,Bird,0.380966,True
1338,birds/close_to_ood/singleBirdinsky102.jpeg,"[Drone, Bird]",True,Bird,0.379062,True
1381,ood_samples/balloons.jpg,"[Drone, Bird]",True,Bird,0.364684,True
1375,ood_samples/a461bb93-1577-4663-a5c4-5652b91b14...,"[Drone, Bird]",True,Bird,0.342295,True
1403,ood_samples/AUjKlvd.jpg,"[Drone, Bird]",True,Bird,0.300620,True
700,drones/close_to_ood/c617f8ac-74db-41a4-8bc1-aa...,"[Drone, Bird]",True,Drone,0.242038,True
1361,bird_drone/4N0W8J7_copyright_image_218654.jpeg,"[Drone, Bird]",True,Bird,0.198754,True
1378,ood_samples/0e1676df-c3e9-44de-aa4a-1e28d23bde...,"[Drone, Bird]",True,Bird,0.187136,True
1393,ood_samples/df543bdd-b483-4ee4-a0f0-aed44d31fa...,"[Drone, Bird]",True,Bird,0.014525,True


In [9]:
data_df[data_df["ood_flag"]]["relative_path"].values[-10:-1]

array(['drones/close_to_ood/e5ed9253-3142-4bc7-8ee3-5e6c8b0e8e45.png',
       'ood_samples/intel-laminar-rm1-03.jpg',
       'ood_samples/fc81b00c-048f-432e-9251-517586852c3e.jpg',
       'bird_drone/4N0W8J7_copyright_image_218654.jpeg',
       'ood_samples/balloons.jpg',
       'ood_samples/0e1676df-c3e9-44de-aa4a-1e28d23bde13.jpg',
       'ood_samples/a461bb93-1577-4663-a5c4-5652b91b14ac.jpg',
       'ood_samples/AUjKlvd.jpg',
       'drones/close_to_ood/c617f8ac-74db-41a4-8bc1-aace9601f01f.jpg'],
      dtype=object)